In [1]:
%load_ext pycodestyle_magic
%load_ext mypy_ipython
%pycodestyle_on

In [2]:
import doctest

In [3]:
import logging
import sys
from functools import wraps, partial

logging.basicConfig(level=logging.DEBUG, stream=sys.stdout)


def attach_wrapper(obj, func=None):
    if func is None:
        return partial(attach_wrapper, obj)

    setattr(obj, func.__name__, func)
    return func


def logged(level, name=None, message=None):
    def decorator(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__

        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return func(*args, **kwargs)

        @attach_wrapper(wrapper)
        def set_level(new_level):
            nonlocal level
            level = new_level

        @attach_wrapper(wrapper)
        def set_message(new_message):
            nonlocal logmsg
            logmsg = new_message

        return wrapper
    return decorator


@logged(logging.DEBUG)
def add(x, y):
    return x + y


@logged(logging.CRITICAL, 'example')
def spam():
    print('spam')


"""

>>> add(2, 3)  # DEBUG:__main__:add
5
>>> add.set_message('add called')
>>> add(2, 3)  # DEBUG:__main__:add called
5
>>> add.set_level(logging.WARNING)
>>> add(2, 3)  # WARNING:__main__:add called
5
"""

doctest.testmod()

DEBUG:__main__:add
DEBUG:__main__:add called


TestResults(failed=0, attempted=5)